In [4]:
import pickle
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization, GlobalMaxPool2D, MaxPooling2D

In [2]:
image_size = 28
num_labels = 10

## Загрузка данных

In [3]:
with open('X_train.pickle', 'rb') as f:
    X_train = pickle.load(f)

with open('y_train.pickle', 'rb') as f:
    y_train = pickle.load(f)
    
with open('X_test.pickle', 'rb') as f:
    X_test = pickle.load(f)

with open('y_test.pickle', 'rb') as f:
    y_test = pickle.load(f)
    
with open('X_valid.pickle', 'rb') as f:
    X_valid = pickle.load(f)

with open('y_valid.pickle', 'rb') as f:
    y_valid = pickle.load(f)

In [6]:
def reformat(dataset, labels):
    dataset = dataset.reshape((dataset.shape[0], 28, 28, 1)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

In [7]:
X_train, y_train = reformat(X_train, y_train)
X_test, y_test = reformat(X_test, y_test)
X_valid, y_valid = reformat(X_valid, y_valid)

In [16]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [17]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=512,
          validation_data=(X_valid, y_valid),
          verbose=1)

Train on 300000 samples, validate on 93163 samples
Epoch 1/10
300000/300000 [==============================] - 178s 593us/step - loss: 0.8853 - accuracy: 0.7722 - val_loss: 0.7707 - val_accuracy: 0.7996
Epoch 2/10
300000/300000 [==============================] - 181s 605us/step - loss: 0.7051 - accuracy: 0.8167 - val_loss: 0.7211 - val_accuracy: 0.8103
Epoch 3/10
300000/300000 [==============================] - 179s 596us/step - loss: 0.6465 - accuracy: 0.8301 - val_loss: 0.6651 - val_accuracy: 0.8218
Epoch 4/10
300000/300000 [==============================] - 178s 594us/step - loss: 0.5791 - accuracy: 0.8446 - val_loss: 0.5928 - val_accuracy: 0.8368
Epoch 5/10
300000/300000 [==============================] - 178s 595us/step - loss: 0.5247 - accuracy: 0.8568 - val_loss: 0.5524 - val_accuracy: 0.8447
Epoch 6/10
300000/300000 [==============================] - 178s 594us/step - loss: 0.4919 - accuracy: 0.8644 - val_loss: 0.5263 - val_accuracy: 0.8527
Epoch 7/10
300000/300000 [===========

In [9]:
model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=512,
          validation_data=(X_valid, y_valid),
          verbose=1)

Train on 300000 samples, validate on 93163 samples
Epoch 1/10
300000/300000 [==============================] - 42s 141us/step - loss: 0.9783 - accuracy: 0.7617 - val_loss: 0.7614 - val_accuracy: 0.8010
Epoch 2/10
300000/300000 [==============================] - 43s 142us/step - loss: 0.6918 - accuracy: 0.8197 - val_loss: 0.7266 - val_accuracy: 0.8091
Epoch 3/10
300000/300000 [==============================] - 43s 142us/step - loss: 0.6661 - accuracy: 0.8250 - val_loss: 0.7067 - val_accuracy: 0.8127
Epoch 4/10
300000/300000 [==============================] - 43s 142us/step - loss: 0.6498 - accuracy: 0.8287 - val_loss: 0.6929 - val_accuracy: 0.8158
Epoch 5/10
300000/300000 [==============================] - 43s 142us/step - loss: 0.6355 - accuracy: 0.8316 - val_loss: 0.6801 - val_accuracy: 0.8172
Epoch 6/10
300000/300000 [==============================] - 43s 142us/step - loss: 0.6215 - accuracy: 0.8347 - val_loss: 0.6657 - val_accuracy: 0.8211o
Epoch 7/10
300000/300000 [================

In [8]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [9]:
adam = Adam(lr=1e-4)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

In [10]:
datagen = ImageDataGenerator(
            rotation_range=10, 
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            zoom_range=0.1)
datagen.fit(X_train)

In [11]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-6)

In [13]:
model.fit(datagen.flow(X_train, y_train, batch_size=512),
          epochs=10,
          validation_data=(X_valid, y_valid),
          callbacks=[reduce_lr],
          verbose=1)

Epoch 1/10
586/586 [==============================] - 64s 109ms/step - loss: 0.3809 - accuracy: 0.8857 - val_loss: 0.3508 - val_accuracy: 0.8955
Epoch 2/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3746 - accuracy: 0.8879 - val_loss: 0.3446 - val_accuracy: 0.8971
Epoch 3/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3689 - accuracy: 0.8893 - val_loss: 0.3414 - val_accuracy: 0.8982
Epoch 4/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3629 - accuracy: 0.8906 - val_loss: 0.3387 - val_accuracy: 0.8987
Epoch 5/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3567 - accuracy: 0.8922 - val_loss: 0.3303 - val_accuracy: 0.9008
Epoch 6/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3519 - accuracy: 0.8942 - val_loss: 0.3297 - val_accuracy: 0.9009
Epoch 7/10
586/586 [==============================] - 63s 108ms/step - loss: 0.3477 - accuracy: 0.8951 - val_loss: 0.3239 - val_ac